In [50]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse
# from scipy.misc import imsave
from tensorflow.keras.applications import vgg19
from tensorflow.keras import backend as K
import os
from PIL import Image, ImageFont, ImageDraw, ImageOps, ImageEnhance, ImageFilter

In [51]:
# precpoces images: reshape, scale
def preprocess_image(image, w, h):
  image = image.resize((w, h))
  image = img_to_array(image)
  image = np.expand_dims(image, axis=0)
  image = vgg19.preprocess_input(image)  # 0-255 ---> 0-1
  return image

In [52]:
def deprocess_image(x, w, h):
  x = x.reshape((w, h, 3))
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  # 'BGR'->'RGB'
  x = x[:, :, ::-1]
  x = np.clip(x, 0, 255).astype('uint8')  # 以防溢出255范围
  return x

In [53]:
def gram_matrix(x):  # Gram矩阵
    assert K.ndim(x) == 3
    if K.image_data_format() == 'channels_first':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [54]:
# 风格损失，是风格图片与结果图片的Gram矩阵之差，并对所有元素求和
# only one layer
# while computing, we must use K because they are tensors, and also we will computer gradient later on.
def style_loss(style, combination, w, h):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    S_C = S-C
    channels = 3
    size = h * w
    return K.sum(K.square(S_C)) / (4. * (channels ** 2) * (size ** 2))
   

In [55]:
# an auxiliary loss function
# designed to maintain the "content" of the
# base image in the generated image
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [56]:
# designed to keep the generated image locally coherent
def total_variation_loss(x,img_nrows, img_ncols):
    assert K.ndim(x) == 4
    if K.image_data_format() == 'channels_first':
        a = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, 1:, :img_ncols - 1])
        b = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, :img_nrows - 1, 1:])
    else:
        a = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
        b = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [57]:
def eval_loss_and_grads(x):  # 输入x，输出对应于x的梯度和loss
    if K.image_data_format() == 'channels_first':
        x = x.reshape((1, 3, height, width))
    else:
        x = x.reshape((1, height, width, 3))
    outs = f_outputs([x])  # 输入x，得到输出
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

In [58]:
picture_size = 256
base_image_path = 'base.jpg'
style_reference_image_path = 'style.jpg'
width = height = 256

In [59]:
base_image = K.variable(preprocess_image(load_img(base_image_path), picture_size, pictrue_size))   
style_reference_image = K.variable(preprocess_image(load_img(style_reference_image_path), picture_size, pictrue_size))
from keras.layers import *
combination_image = Input(shape=(256, 256, 3))
input_tensor = Lambda(lambda combination_image: K.concatenate([base_image, style_reference_image, combination_image], axis=0))(combination_image)   #组合
 
model = vgg19.VGG19(input_tensor=input_tensor,weights='imagenet', include_top=False)
model.summary()

The following Variables were used a Lambda layer's call (lambda_1), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(1, 256, 256, 3) dtype=float32>
  <tf.Variable 'Variable:0' shape=(1, 256, 256, 3) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Co

In [60]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

In [61]:
# loss = K.variable(0.)
# it must be define as a tf variable instead of the general python var

In [62]:
import tensorflow as tf 
with tf.GradientTape() as tape:
    layer_features = outputs_dict['block5_conv2']
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    content_weight = 0.08
    loss = content_weight * content_loss(base_image_features, combination_features)

    feature_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1','block5_conv1']
    feature_layers_w = [0.1,0.1,0.4,0.3,0.1]
    # feature_layers = ['block5_conv1']
    # feature_layers_w = [1]
    for i in range(len(feature_layers)):
        # 每一层的权重以及数据
        layer_name, w = feature_layers[i], feature_layers_w[i]
        layer_features = outputs_dict[layer_name]  # 该层的特征
    
        style_reference_features = layer_features[1, :, :, :]  # 参考图像在VGG网络中第i层的特征
        combination_features = layer_features[2, :, :, :]     # 结果图像在VGG网络中第i层的特征
    
        loss = loss + w * style_loss(style_reference_features, combination_features, 256, 256)  # 目标风格图像的特征和结果图像特征之间的差异作为loss
    
    loss = loss + total_variation_loss(combination_image, 256, 256)
# 求得梯度，输入combination_image，对loss求梯度, 每轮迭代中combination_image会根据梯度方向做调整
grads = tape.gradient(loss, combination_image)
print(grads)
outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)
 
f_outputs = K.function([combination_image], outputs)

None


AttributeError: ignored

In [63]:

layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
content_weight = 0.08
loss = content_weight * content_loss(base_image_features, combination_features)

feature_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1','block5_conv1']
feature_layers_w = [0.1,0.1,0.4,0.3,0.1]
# feature_layers = ['block5_conv1']
# feature_layers_w = [1]
for i in range(len(feature_layers)):
    # 每一层的权重以及数据
    layer_name, w = feature_layers[i], feature_layers_w[i]
    layer_features = outputs_dict[layer_name]  # 该层的特征
    
    style_reference_features = layer_features[1, :, :, :]  # 参考图像在VGG网络中第i层的特征
    combination_features = layer_features[2, :, :, :]     # 结果图像在VGG网络中第i层的特征
    
    loss = loss + w * style_loss(style_reference_features, combination_features, 256, 256)  # 目标风格图像的特征和结果图像特征之间的差异作为loss
    
loss = loss + total_variation_loss(combination_image, 256, 256)
# 求得梯度，输入combination_image，对loss求梯度, 每轮迭代中combination_image会根据梯度方向做调整
grads = K.gradients(loss, combination_image)
print(grads)
outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)
 
f_outputs = K.function([combination_image], outputs)

RuntimeError: ignored

In [ ]:
 
def eval_loss_and_grads(x):  # 输入x，输出对应于x的梯度和loss
    if K.image_data_format() == 'channels_first':
        x = x.reshape((1, 3, height, width))
    else:
        x = x.reshape((1, height, width, 3))
    outs = f_outputs([x])  # 输入x，得到输出
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

In [ ]:
# Evaluator可以只需要进行一次计算就能得到所有的梯度和loss
class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
 
    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
 
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [ ]:
evaluator = Evaluator()
x = preprocess_image(source_image)
img = deprocess_image(x.copy())
fname = '原始图片.png'
save_img(fname, img)


In [ ]:
iterations = 5
# 开始迭代
for i in range(iterations):
    start_time = time.time()
    print('迭代', i,end="   ")
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.grads, maxfun=20, epsilon=1e-7)
    # 一个scipy的L-BFGS优化器
    print('目前loss:', min_val,end="  ")
    # 保存生成的图片
    img = deprocess_image(x.copy())
 
    fname = 'result_%d.png' % i
    end_time = time.time()
    print('耗时%.2f s' % (end_time - start_time))
 
    if i%5 == 0 or i == iterations-1:
        save_img(fname, img, image_enhance=True)
        print('文件保存为', fname)